In [7]:
import langsmith as ls

client = ls.Client()
experiment = "kind-cat-98"
runs = list(client.list_runs(project_name=experiment, is_root=True))
print(f"Found {len(runs)} runs")

Found 1230 runs


In [8]:
from collections import defaultdict

d = defaultdict(list)
for r in runs:
    d[r.reference_example_id].append(r.feedback_stats["quality"]["avg"])

In [15]:
hardest[0][0]

UUID('9268a6ce-4ce0-4e34-8f31-b38595b4f1a0')

In [16]:
import statistics
from collections import Counter

nopass = list(set([k for k, v in d.items() if not any(s == 1.0 for s in v)]))
avg = [(k, statistics.mean(v)) for k, v in d.items()]
hardest = sorted(avg, key=lambda x: x[1])[:150]
hardest_set = {e[0] for e in hardest}
ex = client.read_example(hardest[0][0])
examples = [
    e for e in client.list_examples(dataset_id=ex.dataset_id) if e.id in hardest_set
]
users = list({e.metadata["user_id"] for e in examples})

In [18]:
import random

users = list({e.metadata["user_id"] for e in examples})
random.shuffle(users)
collected = []
cum = 0
splits = (0.3, 0.4, 0.3)
for line in splits:
    start, end = int(len(users) * (cum)), int(len(users) * (cum + line))
    collected.append(users[start:end])
    cum += line
train, dev, test = collected
umap = {
    uid: split
    for split, uids in zip(["train", "dev", "test"], collected)
    for uid in uids
}

0 6
6 14
14 20


In [19]:
splits_examples = defaultdict(list)
for e in examples:
    splits_examples[umap[e.metadata["user_id"]]].append(e)

In [20]:
for k, v in splits_examples.items():
    print(k, Counter([e.metadata["category"] for e in v]))

dev Counter({'marketing': 25, 'hardware/device issues': 21, 'community management': 11, 'bug reports': 1})
train Counter({'marketing': 23, 'hardware/device issues': 22, 'community management': 9, 'bug reports': 2})
test Counter({'marketing': 17, 'community management': 9, 'hardware/device issues': 9, 'bug reports': 1})


In [23]:
new_dataset_name = "CS Simulations"
if client.has_dataset(dataset_name=new_dataset_name):
    client.delete_dataset(dataset_name=new_dataset_name)
ds = client.create_dataset(dataset_name=new_dataset_name)
for split_name, examples in splits_examples.items():
    client.create_examples(
        inputs=[{"messages": e.inputs["messages"]} for e in examples],
        outputs=[e.outputs for e in examples],
        metadata=[e.metadata for e in examples],
        splits=[split_name] * len(examples),
        dataset_id=ds.id,
    )